CÉLULA DE MARKDOWN EXEMPLO

## Projeto IIA ##
* Professor Díbio , Turma 01 , 2025.1

# Autores: #
* Marcelo Marques Rodrigues - 221018960

favor rodar{ pip install -r requirements.txt }

## Parte 1: Processar Geolocalizações e Mostrar em Mapa ##

In [29]:
import os
import pandas as pd
from geopy.distance import geodesic

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import folium
import time


Processamos o Dataset

In [30]:
caminho_arquivo_csv = os.path.join( "..", "data", "associacoes_geocoords.csv" )

try:
    df_associacoes = pd.read_csv(caminho_arquivo_csv)

except FileNotFoundError:
    print('Arquivo .csv não encontrado! Verifique o Diretório')
    df_associacoes = pd.DataFrame( columns=['Nome_Associacao', 'Localidade_Principal_Projeto', 'Latitude', 'Longitude'] )

print("Dataframe Carregado")
print(df_associacoes.head(6))
print(f'\n Número Total de Associações: {len(df_associacoes)}')

Dataframe Carregado
                                     Nome_Associacao  \
0  Associação de Agricultores Familiares da Eco C...   
1  Associação Agrícola do Distrito Federal e Ride...   
2  Associação Mista dos Agricultores Familiares, ...   
3  Associação dos Produtores da Agricultura Famil...   
4  Associação dos Produtores Rurais de Alexandre ...   
5  Associação dos Produtores de Hortigranjeiros d...   

  Localidade_Principal_Projeto  Latitude  Longitude  
0                São Sebastião -15.88007  -47.77402  
1                   Taguatinga -15.83322  -48.05677  
2                  Santa Maria -16.00157  -47.98586  
3                        Guará -15.81459  -47.98497  
4                   Brazlândia -15.67693  -48.20507  
5                         Gama -16.01609  -48.07178  

 Número Total de Associações: 17


Calculamos distância e encontramos as associações mais próximas comparadas à entrada do usuário

In [31]:
def calcular_distancia_km(coord_usuario, coord_associacao): # calculamos a distancia por coordenadas do usuario com a associação

    if None in coord_usuario or pd.isna(coord_usuario[0]) or pd.isna(coord_usuario[1]) or \
       None in coord_associacao or pd.isna(coord_associacao[0]) or pd.isna(coord_associacao[1]):
        return float('inf') # retorna infinito se a coordenada do usuária for none ou nan
    
    return geodesic(coord_usuario, coord_associacao).km

def encontrar_associacoes_proximas( df_associacoes, coord_usuario_lat, coord_usuario_lon, max_distancia_km=None, top_n=None):
    
    if df_associacoes.empty:
        print(" DataFrame vazio. Não há associação para encontrar" )
        return pd.DataFrame()

    coord_usuario = (coord_usuario_lat, coord_usuario_lon)

    df_resultado = df_associacoes.copy()

    df_resultado['Latitude'] = pd.to_numeric(df_resultado['Latitude'], errors='coerce') # tratamento
    df_resultado['Longitude'] = pd.to_numeric(df_resultado['Longitude'], errors='coerce') #tratamento
    df_resultado['Distancia_km'] = df_resultado.apply(
        lambda row: calcular_distancia_km(coord_usuario, (row['Latitude'], row['Longitude'])), # lambda permite calcular todas as dis^tanias
        axis=1
    )

    df_resultado = df_resultado.sort_values(by='Distancia_km') # sort

    if max_distancia_km is not None:
        df_resultado = df_resultado[df_resultado['Distancia_km'] <= max_distancia_km]
    if top_n is not None: # melhores N recomendações
        df_resultado = df_resultado.head(top_n)

    return df_resultado

Teste da função

In [ ]:
def geocodificar_endereco_brasilia(endereco, tentativas=3, delay=2):

    geolocator = Nominatim(user_agent="projeto_iia") 
    endereco_completo = f"{endereco}, Brasília, DF, Brazil"
    print(f"Tentando geocodificar: '{endereco_completo}'")
    for tentativa in range(tentativas):
        try:
            location = geolocator.geocode(endereco_completo, timeout=10) # responder dentro do timeout
            if location:
                print(f"Endereço encontrado: {location.address}")
                return location.latitude, location.longitude
            else:
                print("Endereço não encontrado.")
                return None
        except GeocoderTimedOut:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Timeout na geocodificação. Tentando novamente em {delay}s...")
            time.sleep(delay)
        except GeocoderUnavailable:
            print(f"Tentativa {tentativa + 1} de {tentativas}: Serviço de geocodificação indisponível. Tentando novamente em {delay}s...")
            time.sleep(delay)
        except Exception as e:
            print(f"Um erro inesperado ocorreu durante a geocodificação: {e}")
            return None
    print("Falha na geocodificação após múltiplas tentativas.")
    return None

lat_usuario , lon_usuario = geocodificar_endereco_brasilia('Centro educacional de taguatinga' , 3 , 2)

associacoes_proximas = encontrar_associacoes_proximas(
    df_associacoes,
    lat_usuario,
    lon_usuario,
    max_distancia_km= 20, # raio do filtro
)

if not associacoes_proximas.empty:
    print(f"\n Associações encontradas perto de ({lat_usuario} , {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 
                            'Localidade_Principal_Projeto',
                            'Distancia_km']
    print(associacoes_proximas[colunas_para_mostrar])
else:

    print(f"\nNenhuma associação encontrada dentro dos critérios perto do usuário.")

# sem filtro de distância máxima, apenas as 3 mais próximas
print("\n--------------------------------------------------")
associacoes_topn = encontrar_associacoes_proximas (
    df_associacoes,
    lat_usuario,
    lon_usuario,
    top_n = 3
)

if not associacoes_topn.empty:
    print(f"\n Top 3 associações mais próximas de ({lat_usuario}, {lon_usuario}):")
    colunas_para_mostrar = ['Nome_Associacao', 'Localidade_Principal_Projeto', 'Distancia_km',]
    print(associacoes_topn[colunas_para_mostrar])
else:
    print(f"\nNenhuma associação encontrada perto do usuário.")

Tentando geocodificar: 'Centro educacional de taguatinga, Brasília, DF, Brazil'


Endereço encontrado: Centro Educacional 04 de Taguatinga, Área especial 20, Taguatinga Norte, Taguatinga, Região Geográfica Imediata do Distrito Federal, Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Distrito Federal, Região Centro-Oeste, 72130-005, Brasil

 Associações encontradas perto de (-15.7953977 , -48.0579086):
                                      Nome_Associacao  \
1   Associação Agrícola do Distrito Federal e Ride...   
7   Associação dos Produtores Rurais Novo Horizont...   
6   Associação dos Produtores Rurais Orgânicos e C...   
15  Cooperativa Agrícola da Região de Planaltina (...   
8   Associação dos Trabalhadores Rurais da Agricul...   
3   Associação dos Produtores da Agricultura Famil...   
11  Cooperativa dos Produtores Rurais de Planaltin...   

   Localidade_Principal_Projeto  Distancia_km  
1                    Taguatinga      4.187071  
7                     Ceilândia      6.802690  
6   